<a href="https://colab.research.google.com/github/mpianforini/FloodSformer/blob/main/docs/demo/Demo_FS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FloodSformer: real-time forecasting

This is a demo of the real-time forecasting procedure of the FloodSformer model. Further details can be found in our paper: Pianforini et al. (2024). Real-Time Flood Maps Forecasting for Dam-Break Scenarios with a Transformer-Based Deep Learning Model. *Journal of Hydrology*. \
\
Case studies:
1.   Dam-break in a channel with a parabolic cross section
2.   Dam-break in a rectangular tank

The resulting output maps (target, predicted and differences maps) will be stored in the designated output folder. Users have the capability to download the entire folder for convenient access.

Note: please use a GPU if available.

In [ ]:
import torch
import os
import sys
import urllib.request as req
from datetime import datetime, timedelta

# check whether run in Colab
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    # Intall the packages
    !pip install torchmetrics -q
    !pip install simplejson -q
    !pip install einops -q
    !pip install psutil -q
    !pip install timm -q
    !pip install opencv-python -q
    !pip install -U fvcore -q
    !pip install zenodo-get -q
else:
    sys.path.append('..')
    raise Warning("You are not using Colab. Ensure all necessary packages are installed before proceeding.")

In [ ]:
# Clone the GitHub repository
print('Cloning the GitHub repository')
!git clone https://github.com/mpianforini/FloodSformer.git
sys.path.append('./FloodSformer')
if os.getcwd() != '/content/FloodSformer':
  os.chdir('FloodSformer/')

# Clone the Zenodo repository (dataset and weights)
print('Cloning the Zenodo repository')
!zenodo_get '10.5281/zenodo.10878385'
!unzip -q FloodSformer_datasets&checkpoints.zip
!rm FloodSformer_datasets&checkpoints.zip

***
## Run the real-time forecasting

In [ ]:
### PARAMETERS ###
# Case study:
# 1 - Dam-break in a channel with a parabolic cross section
# 2 - Dam-break in a rectangular tank
case_study = 1

# Number of past frames
past_frames = 8  # P: number of past frames (between 1 and 8)

if case_study == 1:
  chkpt_AE_dir = 'DB_parabolic/checkpoint/AE_parab_e86.tar'
  chkpt_VPTR_dir = 'DB_parabolic/checkpoint/VPTR_parab_e116.tar'
  dataset_dir = 'DB_parabolic/dataset_test/'
  cfg_file = 'configs/Dam_break/DB_parabolic_testFS.yaml'
  future_frames = 106 - past_frames
  name_download = 'DB_parabolic_Results_P{}_F{}.zip'.format(past_frames, future_frames)
  print("Case study 1 - Dam-break in a channel with a parabolic cross section")
  print("Past frames = {}".format(past_frames))
  print("Future frames = {}".format(future_frames))
elif case_study == 2:
  chkpt_AE_dir = 'DB_tank/checkpoint/AE_tank_e66.tar'
  chkpt_VPTR_dir = 'DB_tank/checkpoint/VPTR_tank_e212.tar'
  dataset_dir = 'DB_tank/dataset_test/'
  cfg_file = 'configs/Dam_break/DB_tank_testFS.yaml'
  future_frames = 121 - past_frames
  name_download = 'DB_tank_Results_P{}_F{}.zip'.format(past_frames, future_frames)
  print("Case study 2 - Dam-break in a rectangular tank")
  print("Past frames = {}".format(past_frames))
  print("Future frames = {}".format(future_frames))
else:
  raise NotImplementedError("Case study not implemented")

assert past_frames < 9 and past_frames > 0

In [ ]:
from docs.demo.demo_utils import set_config, run_RTforecast
from floodsformer.config.defaults import _assert_and_infer_cfg

# Setup cfg.
cfg = set_config(cfg_file, past_frames, future_frames, chkpt_AE_dir, chkpt_VPTR_dir, dataset_dir)

if torch.cuda.is_available():
  cfg.NUM_GPUS = 1
  print("Run using GPU")
  print("Num GPUs Available: ", torch.cuda.device_count())
  !nvidia-smi -L
else:
  cfg.NUM_GPUS = 0
  print("WARNING: running using CPU. High computational time!\n")
  print("CPU type:")
  !cat /proc/cpuinfo | grep "model name" | head -1
  print("Number of cores:")
  !cat /proc/cpuinfo | grep "model name" | wc -l

# Make some assertions.
cfg = _assert_and_infer_cfg(cfg)

# Run the real-time forecasting procedure.
save_dir, renorm_transform = run_RTforecast(cfg)
print("Output grids saved in: {}".format(save_dir))

***
## Print ground-truth and predicted maps
- The first row of the figure represents the ground-truth maps obtained with the PARFLOOD code (numerical model).
- The second row reports the maps predicted by the FloodSformer model (surrogate model).
- The last row represents the difference between predicted and ground-truth maps.

In [ ]:
from docs.demo.demo_utils import MapToImage
MapToImage(cfg, save_dir, renorm_transform)

### Plot the RMSE

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = mpimg.imread(os.path.join(save_dir, 'plot_rmse_seq_0.png'))
plt.imshow(img, origin='upper')
plt.axis('off')  # Turn off axis
plt.show()

***
## Download the results
The output grids (ground-truth, predicted and differences) for all the future frames are available for download from the designated output folder. \
Users have the option to manually retrieve the files or utilize the following code to download the results from the most recent run.

In [ ]:
import shutil
from google.colab import files

# Create a zip file of the folder with the results of the last run.
shutil.make_archive(name_download, 'zip', save_dir)
# Download the zip file to local machine
files.download(name_download)
print("{} downloaded.".format(name_download))